Homework 10, part 1

In [3]:
# Make sure nest_asyncio is applied in an earlier cell
import asyncio
from playwright.async_api import async_playwright

async def get_all_fields():
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.dmr.nd.gov/oilgas/findwellsvw.asp", wait_until="networkidle")
        
        # Grab all options from the ddmField dropdown
        options = await page.eval_on_selector_all(
            "#ddmField option",
            "elements => elements.map(el => el.value).filter(v => v.trim() !== '')"
        )
        
        await browser.close()
        return options

# In Jupyter, we can use 'await' directly
all_fields = await get_all_fields()
print(all_fields[:20])  # preview first 20 fields
print(f"Total fields: {len(all_fields)}")


['ALEXANDER', 'ALEXANDRIA', 'ALGER', 'ALKABO', 'ALKALI CREEK', 'AMBROSE', 'AMIDON', 'AMOR', 'ANDERSON COULEE', 'ANTELOPE', 'ANTELOPE CREEK', 'ANTLER', 'ARNEGARD', 'ASH', 'ASH COULEE', 'ASSINIBOINE', 'AURELIA', 'AUSTIN', 'AVOCA', 'BADEN']
Total fields: 610


In [9]:
import asyncio
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from playwright.async_api import async_playwright

all_fields = ["ALEXANDER", "ALEXANDRIA", "ALGER"]  # replace with full list later

async def scrape_all_wells(fields):
    all_data = []

    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=False)
        page = await browser.new_page()
        await page.goto("https://www.dmr.nd.gov/oilgas/findwellsvw.asp", wait_until="networkidle", timeout=60000)

        for field in tqdm(fields, desc="Scraping townships"):
            await page.select_option("#ddmField", field)
            await page.click('input[name="B1"]')
            time.sleep(2)  # wait for table to update

            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            rows = soup.select("tr")[1:]  # skip header

            for row in rows:
                cols = [td.get_text(strip=True) for td in row.find_all("td")]
                if cols:
                    cols.append(field)  # add township
                    all_data.append(cols)

        await browser.close()

    # Create DataFrame without fixed column names
    max_cols = max(len(r) for r in all_data)  # find the widest row
    column_names = [f"Column_{i+1}" for i in range(max_cols-1)] + ["Field"]
    df = pd.DataFrame(all_data, columns=column_names)
    return df

all_wells_df = await scrape_all_wells(all_fields)
all_wells_df.head()


Scraping townships:   0%|          | 0/3 [00:00<?, ?it/s]

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,Column_11,Field
0,"Data Services Notification:January 19, 2022 Da...",ALEXANDER,None,None,None,None,None,None,None,None,None,None
1,32709,221244,3305307628,OG,A,4/29/2023,21459,SESW 7-151-100,"DEVON ENERGY WILLISTON, L.L.C",HEINZ 18-19 XW #1H,ALEXANDER,ALEXANDER
2,40882,,3305310303,OG,PNC,9/25/2024,,LOT1 30-151-100,"CONTINENTAL RESOURCES, INC.",Berlain 7-30HSL,ALEXANDER,ALEXANDER
3,22021,222021,3305303899,OG,A,5/1/2024,20290,SWSW 11-151-101,"DEVON ENERGY WILLISTON, L.L.C",BILL 14-23 2TFH,ALEXANDER,ALEXANDER
4,22023,222021,3305303901,OG,A,5/1/2024,21789,SWSW 11-151-101,"DEVON ENERGY WILLISTON, L.L.C",BILL 14-23 1H,ALEXANDER,ALEXANDER


In [11]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from playwright.async_api import async_playwright
import time
import asyncio

# Full list of North Dakota townships (you can expand this)
all_fields = [
    "ALEXANDER","ALEXANDRIA","ALGER","ALKABO","ALKALI CREEK","AMBROSE","AMIDON",
    # ... add all field names you want ...
    "ZENITH","ZION"
]

# Expected columns
columns = [
    "Well Number","Other ID","API Number","Type","Status","Date",
    "Depth","Location","Operator","Lease/Well Name","Field"
]

async def scrape_nd_wells():
    all_data = []

    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=False)
        page = await browser.new_page()
        url = "https://www.dmr.nd.gov/oilgas/findwellsvw.asp"

        for field in tqdm(all_fields, desc="Scraping ND Townships"):
            print(f"Processing: {field}")

            await page.goto(url, wait_until="networkidle", timeout=60000)

            # Select field/township
            await page.select_option("select[name='ddmField']", field)
            await page.click("input[name='B1']")

            time.sleep(2)  # give the table time to update

            # Wait for table rows
            try:
                await page.wait_for_selector("tr", timeout=10000)
            except:
                print(f"No data for {field}")
                continue

            # Get HTML
            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            rows = soup.select("tr")[1:]  # skip header row

            for row in rows:
                cells = [td.get_text(strip=True) for td in row.find_all("td")]

                if not cells:
                    continue

                # Force row length to match columns
                if len(cells) < len(columns):
                    cells += [""] * (len(columns) - len(cells))
                elif len(cells) > len(columns):
                    cells = cells[:len(columns)]

                # Replace the last column with the field name
                cells[-1] = field  

                all_data.append(cells)

        await browser.close()

    # Save to DataFrame
    df = pd.DataFrame(all_data, columns=columns)
    df.to_csv("ND_Wells.csv", index=False)

    print("✔ Scraping complete!")
    print("✔ Saved to ND_Wells.csv")

# Jupyter: run using await (NOT asyncio.run)
await scrape_nd_wells()


Scraping ND Townships:   0%|          | 0/9 [00:00<?, ?it/s]

Processing: ALEXANDER
Processing: ALEXANDRIA
Processing: ALGER
Processing: ALKABO
Processing: ALKALI CREEK
Processing: AMBROSE
Processing: AMIDON
Processing: ZENITH
Processing: ZION
✔ Scraping complete!
✔ Saved to ND_Wells.csv
